In [1]:
import aiida
from ase.build import bulk
from aiida.orm import StructureData, Str, load_code, load_group
from aiida.plugins import CalculationFactory, DataFactory

## Using AiiDA to generate input file

In [2]:
PwCalculation = CalculationFactory("quantumespresso.pw")
Dict = DataFactory("core.dict")
KpointsData = DataFactory("core.array.kpoints")

In [3]:
aiida.load_profile()

Profile<uuid='d2d76748653d48f58fc3c3b14174d426' name='aiida_default'>

In [4]:
pseudo_family = load_group('SSSP/1.3/PBEsol/efficiency')

In [5]:
code = load_code('pw@qe-computer')

In [6]:
structure = StructureData(ase=bulk('C', 'diamond', 2.0))

In [7]:
parameters = Dict({
        'CONTROL': {
            'calculation': 'scf',
            'verbosity': 'high',
            'tstress': True,
            'tprnfor': True,
        },
        'SYSTEM': {
            'ecutwfc': 30,
            'ecutrho': 8*30,
            'occupations': 'smearing', # options are 'smearing', 'tetrahedra', 'tetrahedra_opt', 'fixed', 'from_input'
            'degauss': 0.0045, # in Ry, 1 Ry = 13.6056981 eV
            'smearing': 'fermi-dirac', # options are 'gaussian', 'methfessel-paxton', 'marzari-vanderbilt', 'fermi-dirac'
        },
        'ELECTRONS':{
            'conv_thr': 1e-8,
            'diagonalization': 'ppcg', # options are 'david', 'cg', 'ppcg', 'paro', 'rmm-davidson'
            'mixing_mode': 'plain', # options are 'plain', 'TF', 'local-TF'
            'mixing_beta': 0.6, 
            'startingwfc':'atomic+random' # options are 'atomic','atomic+random', 'random','file'
        }
})

kpoints = KpointsData()
klist=[4,4,4]
kpoints.set_kpoints_mesh(klist)
builder = PwCalculation.get_builder()
builder.code = code
builder.structure = structure
builder.kpoints = kpoints
builder.parameters = Dict(dict=parameters)
builder.pseudos = pseudo_family.get_pseudos(structure=structure)
builder.metadata.options.resources = {"num_machines": 1, "num_mpiprocs_per_machine": 8}
builder.metadata.options.withmpi = True
builder.metadata.options.max_wallclock_seconds = 30 * 60

In [12]:
from aiida.plugins import CalculationFactory
from aiida.engine import Process
from aiida.common.folders import SandboxFolder

# Use the PwCalculation from the Quantum ESPRESSO plugin
# PwCalculation = CalculationFactory('quantumespresso.pw')

# Create a PwCalculation instance with the builder's inputs
process = PwCalculation(inputs=builder._inputs(prune=True))

# Use a sandbox folder to hold the temporary input files
with SandboxFolder() as folder:
    # Call prepare_for_submission on the process instance
    calc_info = process.prepare_for_submission(folder)

    # The main input file is typically named 'aiida.in'
    input_file_path = folder.get_abs_path('aiida.in')

    # Read and print the content of the input file
    with open(input_file_path, 'r') as f:
        input_file_content = f.read()

    print(input_file_content)


11/07/2024 04:06:10 PM <7446> aiida.broker.rabbitmq: [WARNING] RabbitMQ v4.0.2 is not supported and will cause unexpected problems!
11/07/2024 04:06:10 PM <7446> aiida.broker.rabbitmq: [WARNING] It can cause long-running workflows to crash and jobs to be submitted multiple times.
11/07/2024 04:06:10 PM <7446> aiida.broker.rabbitmq: [WARNING] See https://github.com/aiidateam/aiida-core/wiki/RabbitMQ-version-to-use for details.


&CONTROL
  calculation = 'scf'
  outdir = './out/'
  prefix = 'aiida'
  pseudo_dir = './pseudo/'
  tprnfor = .true.
  tstress = .true.
  verbosity = 'high'
/
&SYSTEM
  degauss =   4.5000000000d-03
  ecutrho = 240
  ecutwfc = 30
  ibrav = 0
  nat = 2
  ntyp = 1
  occupations = 'smearing'
  smearing = 'fermi-dirac'
/
&ELECTRONS
  conv_thr =   1.0000000000d-08
  diagonalization = 'ppcg'
  mixing_beta =   6.0000000000d-01
  mixing_mode = 'plain'
  startingwfc = 'atomic+random'
/
ATOMIC_SPECIES
C      12.011 C.pbesol-n-kjpaw_psl.1.0.0.UPF
ATOMIC_POSITIONS angstrom
C            0.0000000000       0.0000000000       0.0000000000
C            0.5000000000       0.5000000000       0.5000000000
K_POINTS automatic
4 4 4 0 0 0
CELL_PARAMETERS angstrom
      0.0000000000       1.0000000000       1.0000000000
      1.0000000000       0.0000000000       1.0000000000
      1.0000000000       1.0000000000       0.0000000000



In [13]:
from aiida.orm import load_code
from aiida.engine import run_get_node
from aiida.plugins import CalculationFactory
from aiida_quantumespresso.calculations.pw import PwCalculation

# Load code (assuming 'pw@qe-computer' is already set up in your AiiDA environment)
code = load_code('pw@qe-computer')
builder.code = code

# Set dry_run mode so that it prepares inputs without executing
builder.metadata.dry_run = True
builder.metadata.store_provenance = False  # Skip provenance storage for this test

# Run the calculation in dry run mode to get the inputs
_, calc_node = run_get_node(builder)

# Retrieve the generated input file content from the temporary folder in the repository
input_file_path = calc_node.base.repository.get_object_content('aiida.in')
print(input_file_path)


&CONTROL
  calculation = 'scf'
  outdir = './out/'
  prefix = 'aiida'
  pseudo_dir = './pseudo/'
  tprnfor = .true.
  tstress = .true.
  verbosity = 'high'
/
&SYSTEM
  degauss =   4.5000000000d-03
  ecutrho = 240
  ecutwfc = 30
  ibrav = 0
  nat = 2
  ntyp = 1
  occupations = 'smearing'
  smearing = 'fermi-dirac'
/
&ELECTRONS
  conv_thr =   1.0000000000d-08
  diagonalization = 'ppcg'
  mixing_beta =   6.0000000000d-01
  mixing_mode = 'plain'
  startingwfc = 'atomic+random'
/
ATOMIC_SPECIES
C      12.011 C.pbesol-n-kjpaw_psl.1.0.0.UPF
ATOMIC_POSITIONS angstrom
C            0.0000000000       0.0000000000       0.0000000000
C            0.5000000000       0.5000000000       0.5000000000
K_POINTS automatic
4 4 4 0 0 0
CELL_PARAMETERS angstrom
      0.0000000000       1.0000000000       1.0000000000
      1.0000000000       0.0000000000       1.0000000000
      1.0000000000       1.0000000000       0.0000000000



## Using ASE to generate input file

In [16]:
import numpy as np
import pandas as pd
from ase import Atoms
from ase.io import read
import ase
from ase.calculators.espresso import Espresso, EspressoProfile

In [17]:
v = read('/Users/elena.patyukova/Documents/DFT-data/structures/structures_1/V.cif')

In [18]:
profile = EspressoProfile(
    command='mpirun -n 8 /Users/elena.patyukova/Documents/github/q-e/bin/pw.x',
    pseudo_dir='/Users/elena.patyukova/Documents/github/LLM-DFT-input-generation/pseudos/SSSP_1.3.0_PBEsol_precision/',
)

In [19]:
pseudopotentials = {'V': 'v_pbesol_v1.4.uspp.F.UPF'}

In [20]:
input_data= {
    'system': {'ecutwfc': 40, 'ecutrho': 320, 'smearing': 'fermi-dirac', 'degauss': 0.005, 'occupations': 'smearing'}
}

In [21]:
ase.io.espresso.write_espresso_in('qe.in', v, input_data=input_data, \
                                  pseudopotentials=pseudopotentials, kspacing=0.1, \
                                  koffset=(0, 0, 0))

In [23]:
with open('qe.in','r') as f:
    data=f.read()

In [24]:
print(data)

&CONTROL
/
&SYSTEM
   ecutwfc          = 40
   ecutrho          = 320
   occupations      = 'smearing'
   degauss          = 0.005
   smearing         = 'fermi-dirac'
   ntyp             = 1
   nat              = 1
   ibrav            = 0
/
&ELECTRONS
/
&IONS
/
&CELL
/
&FCP
/
&RISM
/
ATOMIC_SPECIES
V 50.9415 v_pbesol_v1.4.uspp.F.UPF

K_POINTS automatic
5 5 5  0 0 0

CELL_PARAMETERS angstrom
2.58283361000000 0.00000000000000 0.00000000000000
-0.86094419413676 2.43511883180700 0.00000000000000
-0.86094488321594 -1.21755983372948 2.10887445523797

ATOMIC_POSITIONS angstrom
V 0.0000000000 0.0000000000 0.0000000000  


